<div style="border-radius:10px; border:#D0C2F0 solid; padding: 15px; background-color: #F8F1E8; font-size:150%; text-align:left">

<h3 align="left"><font color='#5E5273'>📄 Veri Setinin Hikayesi </font></h3>
    <center><img src="https://i.imgur.com/i14GjZ8.png"> </center>
    
*   Veri MyAnimeList sitesinden alınmıştır, genellikle MAL olarak kısaltılan, gönüllüler tarafından işletilen bir anime ve manga sosyal ağ ve kataloglama uygulama web sitesidir. Site, kullanıcılarına anime ve manga listelerini düzenlemek ve puanlamak için bir liste benzeri sistem sunar.
*   Site adresi : <a href="https://myanimelist.net/"> My Anime List </a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/kaggle/input/anime-recommendation-database-2020/anime.csv")
df.head(5).T

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>👀 Features: </font></h3>
    

1. **MAL_ID:** MyAnimeList ID'si.
2. **Name:** Anime'nin adı.
3. **Score:** Anime'nin genel puanı.
4. **Genres:** Anime'nin türleri.
5. **English name:** Anime'nin İngilizce adı.
6. **Japanese name:** Anime'nin Japonca adı.
7. **Type:** Anime'nin türü (örneğin, TV, Movie, OVA, vs.).
8. **Episodes:** Anime'nin bölüm sayısı.
9. **Aired:** Yayınlanma tarihi.
10. **Premiered:** İlk gösterim tarihi.
11. **Producers:** Yapımcı şirketler.
12. **Licensors:** Lisans sahipleri.
13. **Studios:** Anime'nin yapıldığı stüdyo.
14. **Source:** Anime'nin kaynak malzemesi (örneğin, manga, light novel).
15. **Duration:** Bölüm başına süre.
16. **Rating:** Yaş sınıflandırması.
17. **Ranked:** Genel sıralama.
18. **Popularity:** Popülerlik sıralaması.
19. **Members:** MyAnimeList üyeleri tarafından izleyenlerin sayısı.
20. **Favorites:** Favorilere eklenenlerin sayısı.
21. **Watching:** Şu anda izleyenlerin sayısı.
22. **Completed:** Tamamlayanların sayısı.
23. **On-Hold:** Beklemede olanların sayısı.
24. **Dropped:** Bırakanların sayısı.
25. **Plan to Watch:** İzleme planında olanların sayısı.
26. **Score-10** ile **Score-1:** Kullanıcıların verdiği puan dağılımı. Örneğin, "Score-10" kullanıcıların kaç tanesinin 10 puan verdiğini gösterir.


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Scorelarına Göre Sıralama</p>

In [ ]:
df.columns[df.columns.str.contains("Score")].to_list()

In [ ]:
Score_columns = df.columns[df.columns.str.contains("Score")].to_list()
scores = df[["MAL_ID","Name","Aired"] + Score_columns]
scores.head(10)

In [ ]:
scores.isnull().sum()

In [ ]:
scores = scores.replace("Unknown",np.NaN)

In [ ]:
scores.shape

In [ ]:
scores = scores.dropna(subset=Score_columns[1:])

In [ ]:
scores.shape

In [ ]:
scores.dtypes

In [ ]:
Score_columns

In [ ]:
scores[Score_columns] = scores[Score_columns].astype("float")

In [ ]:
scores.dtypes

In [ ]:
Score_columns[1:]

In [ ]:
def score_calculate(data ,list_of_score_cols):
    list_of_score_cols.reverse()
    final_score = 0
    for i,score_count in enumerate(list_of_score_cols,1):
        final_score += (i * data[score_count])/data[list_of_score_cols].sum(axis = 1)
    return final_score    

In [ ]:
scores["self_score"] = score_calculate(scores,Score_columns[1:])
scores.head(10)

In [ ]:
scores.sort_values(by="self_score",ascending=False).head(10)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Basit, scorları sort_values ile sıralamadır.

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Bayesian Avarage Scorelarına Göre Sıralama</p>

In [ ]:
scores.head(10)

In [ ]:
def bayesian_average_rating(n, confidence=0.95):
    import scipy.stats as st
    import math
    
    if sum(n) == 0: # Satırdaki değerlerin toplamı 0 mı
        return 0
    
    K = len(n) # verilen listenin uzunluğu 10
    
    z = st.norm.ppf(1 - (1 - confidence) / 2) # confidence değerinin Z-skoru, ppf(0.975) https://www.math.arizona.edu/~rsims/ma464/standardnormaltable.pdf
    
    N = sum(n)
    first_part = 0.0
    second_part = 0.0
    for k, n_k in enumerate(n):
        first_part += (k + 1) * (n_k + 1) / (N + K)
        second_part += (k + 1) * (k + 1) * (n_k + 1) / (N + K)
    score = first_part - z * math.sqrt((second_part - first_part * first_part) / (N + K + 1))
    return score

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

- `n`: Gözlemleri içeren bir liste. Her bir eleman, belirli bir derece puanındaki gözlemlerin sayısını temsil eder.
- `confidence`: Güven aralığı. Varsayılan değeri 0.95'tir.

Fonksiyonun işleyişi:

1. Eğer gözlemler toplamı sıfırsa, fonksiyon 0 döner.
2. `K` değişkeni, derecelendirme puanlarının toplam sayısını temsil eder.
3. `z`, güven aralığının Z-skoru olarak adlandırılır ve `scipy.stats` kütüphanesinden elde edilir.
4. `N`, gözlemlerin toplam sayısını temsil eder.
5. `first_part` ve `second_part`, Bayesian ortalama formülündeki toplam terimleri hesaplar.
6. Son olarak, Bayesian ortalama puanı `score` hesaplanır.
    
    <center>
        <img src="https://i.imgur.com/Ksct9mU.png" > 
        <img src="https://i.imgur.com/wjvEh9F.png" > 
        <img src="https://i.imgur.com/qQXuhR9.png" > 
    </center>
    

In [ ]:
Score_columns = df.columns[df.columns.str.contains("Score")].to_list()
Score_columns.reverse()
Score_columns.pop()
Score_columns

In [ ]:
scores["bar_score"] = scores.apply(lambda x: bayesian_average_rating(x[Score_columns]), axis=1)

scores.sort_values(by="bar_score",ascending=False).head(10)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Bayesyen ortalama hesaplama yöntemi, puan dağılımları üzerinde belirli bir güven aralığına göre ağırlıklı bir ortalama yapmamızı sağlar, sadece puanların dağılımına odaklanır.

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Parametre Ağırlıklarına Göre Sıralama</p>

In [ ]:
parametres = df[["MAL_ID","Name","Aired","Score","Members","Favorites","Watching","Completed"]]
parametres["Score"] = scores["self_score"]
parametres.head(10)

In [ ]:
parametres.isnull().sum()

In [ ]:
parametres = parametres.replace("Unknown",np.NaN)

In [ ]:
parametres.isnull().sum()

In [ ]:
parametres = parametres.dropna(subset="Score")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scale(data, cols,min_range, max_range):
    for col in cols:
        data.loc[:,f"{col}_scaled"] = MinMaxScaler(feature_range=(min_range, max_range)).fit_transform(data[[col]])

In [ ]:
scale(parametres,["Members","Favorites", "Watching", "Completed"], 1, 10)
parametres.head(10)

In [ ]:
parametres.dtypes

In [ ]:
parametres.sort_values(by="Score", ascending=False)

In [ ]:
parametres["p_score"] = (parametres["Score"] * 20/100) + \
                        (parametres["Members_scaled"] * 15/100) + \
                        (parametres["Favorites_scaled"] * 25/100) + \
                        (parametres["Watching_scaled"] * 15/100) + \
                        (parametres["Completed_scaled"] * 25/100)


parametres.sort_values(by="p_score", ascending=False)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Ürünleri, sadece verilen puanlar değil, populerlik parametrelerinide göz önünde bulundurarak puanlıyabiliriz, bu sayede sadece verilen puana bağlı bir sıralama yapmaktan kaçınmış oluruz.

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Oy Sayısına Göre Ortalamaya Yaklaştırma</p>

In [ ]:
scores

In [ ]:
def mean_rating(score, score_count, threshold):
    total_count = score_count.sum(axis =1)
    
    all_votes_mean = score.mean()
    
    return (total_count / (total_count + threshold) * score) + (threshold / (total_count + threshold) * all_votes_mean)

In [ ]:
Score_columns = df.columns[df.columns.str.contains("Score")].to_list()
Score_columns

In [ ]:
scores[Score_columns[1:]].sum(axis=1)

In [ ]:
scores["self_score"].mean()

In [ ]:
scores["m_score"] = mean_rating(scores["self_score"],scores[Score_columns[1:]], 1595)

scores

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Bir ürünü, belirli bir oy sayısı parametresi belirleyerek, bu oy sayısından az oy alan ürünleri daha fazla ortalamaya yaklaştırarak, çok oy alan ürünleri ise bu hesaptan minimum etkilenen şekilde yeniden puanlandırabiliriz.